# Map Wells

This notebook demonstrates how to create maps showing well locations using GeoSuite's geospatial capabilities.

## Overview

GeoSuite provides geospatial tools for:
- Loading well location data with coordinates
- Creating maps with well locations
- Visualizing well attributes (status, operator, depth)
- Spatial analysis and proximity calculations

This notebook will show you how to:

1. Load well location data
2. Create maps with well locations
3. Visualize well attributes on maps

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Try to import geospatial demo data
try:
    from geosuite.geospatial.loaders.demo import create_demo_texas_data
    GEOSPATIAL_AVAILABLE = True
except ImportError:
    GEOSPATIAL_AVAILABLE = False
    print("Geospatial module not available. Creating synthetic well data.")

print("GeoSuite imported successfully!")

## 1. Load Well Location Data

Load well data with geographic coordinates (latitude, longitude).

In [ ]:
# Load demo well data with locations
if GEOSPATIAL_AVAILABLE:
    wells, pipelines, facilities, incidents = create_demo_texas_data()
    print(f"Loaded {len(wells)} wells")
    print(f"\nColumns: {wells.columns.tolist()}")
else:
    # Create synthetic well data
    np.random.seed(42)
    n_wells = 100
    wells = pd.DataFrame({
        'well_id': [f'Well_{i+1}' for i in range(n_wells)],
        'name': [f'Well_{i+1}' for i in range(n_wells)],
        'lon': np.random.uniform(-103.5, -101.0, n_wells),  # West Texas
        'lat': np.random.uniform(31.0, 33.0, n_wells),
        'operator': np.random.choice(['Operator_A', 'Operator_B', 'Operator_C'], n_wells),
        'status': np.random.choice(['Active', 'Inactive', 'Plugged'], n_wells, p=[0.7, 0.2, 0.1]),
        'depth_ft': np.random.uniform(5000, 15000, n_wells).astype(int)
    })
    print(f"Created {len(wells)} synthetic wells")

print(f"\nWell data preview:")
wells.head()

## 2. Create Basic Well Map

Create a simple map showing well locations.

In [ ]:
# Create basic well location map
fig, ax = plt.subplots(figsize=(10, 8))

# Plot well locations
ax.scatter(wells['lon'], wells['lat'], s=50, alpha=0.6, c='blue', edgecolors='black', linewidth=0.5)

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title(f'Well Locations Map - {len(wells)} Wells')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

## 3. Color Wells by Status

Visualize well status (Active, Inactive, Plugged) using different colors.

In [ ]:
# Color wells by status
if 'status' in wells.columns:
    status_colors = {'Active': 'green', 'Inactive': 'orange', 'Plugged': 'red'}
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for status, color in status_colors.items():
        status_wells = wells[wells['status'] == status]
        if len(status_wells) > 0:
            ax.scatter(status_wells['lon'], status_wells['lat'], 
                      s=50, alpha=0.6, c=color, label=status, 
                      edgecolors='black', linewidth=0.5)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Well Locations - Colored by Status')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    plt.show()
    
    # Show status distribution
    print("\nWell Status Distribution:")
    print(wells['status'].value_counts())

## 4. Size Wells by Depth

Use marker size to represent well depth.

In [ ]:
# Size wells by depth
if 'depth_ft' in wells.columns:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Normalize depth for marker size
    depth_normalized = (wells['depth_ft'] - wells['depth_ft'].min()) / (wells['depth_ft'].max() - wells['depth_ft'].min())
    sizes = 50 + depth_normalized * 200  # Size range: 50-250
    
    scatter = ax.scatter(wells['lon'], wells['lat'], s=sizes, 
                        alpha=0.6, c=wells['depth_ft'], cmap='viridis',
                        edgecolors='black', linewidth=0.5)
    
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Depth (ft)')
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Well Locations - Sized and Colored by Depth')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    plt.show()

## 5. Summary

This notebook demonstrated:

-  Loading well location data with coordinates
-  Creating basic well location maps
-  Visualizing well attributes (status, depth) on maps

### Next Steps

- Use Apache Sedona for advanced spatial analysis (if available)
- Add basemaps using contextily or folium
- Perform spatial joins with pipelines, facilities, or other features
- Calculate well density using H3 indexing
- Export maps to GeoJSON or other formats